In [27]:
!pip install openai==0.28

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.57.2
    Uninstalling openai-1.57.2:
      Successfully uninstalled openai-1.57.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.2.12 requires openai<2.0.0,>=1.55.3, but you have openai 0.28.0 which is incompatible.
pyragnarok 0.0.1 requires openai>=1.30.5, but you have openai 0.28.0 which is incompatible.
vllm 0.6.4.post1 requires openai>=1.45.0, but you have openai 0.28.0 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# RAG pipeline for Legal Due-diligence M&A documents

### Import necessary libraries for data handling, embeddings, similarity computations, and API calls to OpenAI.

In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

In [2]:
import openai
import pandas as pd
from tqdm import tqdm
import time
from sentence_transformers import SentenceTransformer
from tqdm import tqdm 
import random

/Users/mdwivedi/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Load the DATASET

In [3]:
# Read the dataset
topic = 1086  # Define your topic variable or use a placeholder
data = pd.read_parquet(f'./passage_data/passage_level_data_with_overlapping_{topic}.parquet')
data.head()

,modified_passage,label,mod_char_len,topic
0,"Exhibit 10.1 EXECUTION VERSION $485,000,000 C...",B,3516,1086
1,ION 6.08 Compliance with Material Laws SECTION...,B,3685,1086
2,xisting Liens 7.02 Existing Investments 7.03 E...,B,3474,1086
3,"low #C# "" Acceptable Discount "" has the meanin...",B,3485,1086
4,"intermediaries , Controls or is Controlled by ...",B,3334,1086


### DATA is huge in size and also licenced. So can not show here. Working on sample data

In [4]:
zeros = data[data['label'] == 'B']
ones = data[data['label'] == '1']
sample_zeros = zeros.sample(100, random_state=30)

sample_data = pd.concat([sample_zeros, ones], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)
print(sample_data.shape)
sample_data.head()
    

(219, 4)


,modified_passage,label,mod_char_len,topic
0,after funding ) on the Closing Date specifying...,1,3401,1086
1,nder or any commercial institution satisfying ...,B,3372,1086
2,est rate or fee hereunder shall be conclusive ...,1,3259,1086
3,"date , Type ( if applicable ) , amount and mat...",1,3060,1086
4,") Deposit Accounts , Electronic Chattel Paper...",B,3134,1086


#### Clean the data

In [5]:
# Preprocessing function
def preprocess_text(text):
    # Remove unwanted characters
    text = text.replace("#C#", "").strip()
    # Normalize spacing and punctuation
    text = " ".join(text.split())
    return text

# Apply preprocessing
sample_data["cleaned_passage"] = sample_data["modified_passage"].apply(preprocess_text)
print(sample_data[["cleaned_passage", "label", "topic"]])

                                       cleaned_passage label  topic
0    after funding ) on the Closing Date specifying...     1   1086
1    nder or any commercial institution satisfying ...     B   1086
2    est rate or fee hereunder shall be conclusive ...     1   1086
3    date , Type ( if applicable ) , amount and mat...     1   1086
4    ) Deposit Accounts , Electronic Chattel Paper ...     B   1086
..                                                 ...   ...    ...
214  ayments of principal of the Term Loan shall re...     1   1086
215  f this Section . Each Participant agrees to be...     B   1086
216  which the certificate evidencing such computat...     B   1086
217  the purchases and sales by such Lender of part...     1   1086
218  where in the Loan Documents shal not apply wit...     1   1086

[219 rows x 3 columns]


### Embedding Genration: You can also use OpenAI embedding generation

In [7]:
 # Load a pre-trained Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Enable progress bar
tqdm.pandas()

# Generate embeddings for the cleaned passages with progress bar
sample_data["embeddings"] = sample_data["cleaned_passage"].progress_apply(lambda x: model.encode(x))

# Example: print the first embedding
print(sample_data["embeddings"].iloc[0])

100%|█████████████████████████████████████████| 219/219 [00:09<00:00, 21.92it/s]

[ 2.31927577e-02 -2.46506352e-02 -1.29306512e-02 -8.51152278e-03
 -1.09648677e-02 -8.64852965e-03  3.06400806e-02  1.06322207e-02
  3.78217474e-02  4.00500819e-02  6.90262169e-02 -3.25891413e-02
 -4.63192128e-02 -6.56007184e-03 -3.68485898e-02  6.05104081e-02
 -8.30229651e-03 -3.86253633e-02 -5.17657287e-02  6.16331510e-02
  1.27169684e-01 -5.32141291e-02 -3.73651111e-03  5.51545108e-03
  1.74538288e-02 -2.87904441e-02  5.89907877e-02 -5.90526359e-03
  4.28410359e-02  3.76238488e-02 -8.38205591e-03  1.05954194e-02
  7.83458054e-02 -3.75164002e-02  3.06498036e-02  4.95666899e-02
  1.50744310e-02  2.27196775e-02  2.69007999e-02  5.71823493e-03
  2.22165808e-02 -3.12218592e-02  1.15293134e-02  2.25524200e-04
 -6.41522184e-02  2.69386377e-02  8.83344933e-03  4.71423008e-02
 -8.84262696e-02  5.49327880e-02 -5.20239249e-02 -3.06607373e-02
 -7.72719085e-03  5.50159737e-02 -2.27647256e-02  1.28251677e-02
  4.65412177e-02 -4.05782498e-02 -6.46386668e-02 -7.40000699e-03
 -6.74363319e-03 -4.11058

#### Store the Embeddings

In [8]:
import faiss
import numpy as np

# Convert embeddings to a numpy array
embedding_matrix = np.vstack(sample_data["embeddings"])

# Create and index with FAISS
dimension = embedding_matrix.shape[1]  # Embedding dimension
index = faiss.IndexFlatL2(dimension)   # L2 distance metric
index.add(embedding_matrix)            # Add embeddings to the index

# Save the FAISS index
faiss.write_index(index, "faiss_index_1086")

### Generating Queries from GPT

### Since, we did not have queries for the dataset, so first, we will clustere the risky sentences, then generate the one query per cluster. 

#### Clustering

In [9]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# Filter risky passages
risky_data = sample_data[sample_data['label'] == '1']

# Convert embeddings to a numpy array
embedding_matrix = np.vstack(risky_data["embeddings"])

In [10]:
embedding_matrix

array([[ 0.02319276, -0.02465064, -0.01293065, ...,  0.01557614,
        -0.03719576, -0.08860549],
       [-0.06431529,  0.00539678, -0.04915885, ...,  0.02383379,
        -0.05021168, -0.10217438],
       [-0.09562206,  0.02552925, -0.04925814, ...,  0.01886895,
        -0.02823679, -0.1048939 ],
       ...,
       [-0.04469597, -0.02036622, -0.04697559, ...,  0.00567595,
         0.01464895, -0.07826757],
       [-0.09136903,  0.03798277, -0.06876964, ...,  0.00981596,
        -0.03842178, -0.08457632],
       [-0.0410451 , -0.05131297,  0.02114981, ...,  0.04074522,
        -0.01674086, -0.08560471]], dtype=float32)

In [11]:
from scipy.cluster.vq import kmeans, vq
num_clusters = 5
# Perform k-means clustering
centroids, distortion = kmeans(embedding_matrix, num_clusters)
cluster_ids, _ = vq(embedding_matrix, centroids)
risky_data["cluster_id"] = cluster_ids

/var/folders/ll/hqtjh2m5481_qkt_4p9g5v940000gn/T/ipykernel_73146/452152932.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  risky_data["cluster_id"] = cluster_ids


In [12]:
risky_data.head()

,modified_passage,label,mod_char_len,topic,cleaned_passage,embeddings,cluster_id
0,after funding ) on the Closing Date specifying...,1,3401,1086,after funding ) on the Closing Date specifying...,"[0.023192758, -0.024650635, -0.012930651, -0.0...",2
2,est rate or fee hereunder shall be conclusive ...,1,3259,1086,est rate or fee hereunder shall be conclusive ...,"[-0.06431529, 0.0053967778, -0.04915885, 0.023...",3
3,"date , Type ( if applicable ) , amount and mat...",1,3060,1086,"date , Type ( if applicable ) , amount and mat...","[-0.095622055, 0.02552925, -0.049258143, -0.02...",4
5,igated to make any Swing Loan at a time when a...,1,3413,1086,igated to make any Swing Loan at a time when a...,"[-0.048400756, -0.044329226, -0.026442688, -0....",2
6,dvise each Lender of the details thereof and o...,1,3430,1086,dvise each Lender of the details thereof and o...,"[-0.036808092, -0.043166127, 0.05696466, -0.06...",0


In [13]:
# Display clustered data
print(risky_data[["cleaned_passage", "cluster_id"]])

                                       cleaned_passage  cluster_id
0    after funding ) on the Closing Date specifying...           2
2    est rate or fee hereunder shall be conclusive ...           3
3    date , Type ( if applicable ) , amount and mat...           4
5    igated to make any Swing Loan at a time when a...           2
6    dvise each Lender of the details thereof and o...           0
..                                                 ...         ...
210  th their respective Commitments of such Class ...           4
212  at the Payment Office the aggregate of the amo...           4
214  ayments of principal of the Term Loan shall re...           2
217  the purchases and sales by such Lender of part...           4
218  where in the Loan Documents shal not apply wit...           2

[119 rows x 2 columns]


#### Identify the most representative sentence for each cluster to feed in the LLM

In [15]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Find the most representative sentence for each cluster
cluster_queries = {}
for cluster_id in range(num_clusters):
    # Get embeddings for the current cluster
    cluster_embeddings = embedding_matrix[risky_data["cluster_id"] == cluster_id]
    
    # Get the centroid for the current cluster
    centroid = centroids[cluster_id].reshape(1, -1)
    
    # Calculate cosine similarities
    similarities = cosine_similarity(cluster_embeddings, centroid)
    
    # Find the index of the most representative sentence (highest similarity)
    most_representative_idx = np.argmax(similarities)
    
    # Retrieve the most representative sentence for the cluster
    representative_sentence = risky_data[risky_data["cluster_id"] == cluster_id]["cleaned_passage"].iloc[most_representative_idx]
    cluster_queries[cluster_id] = representative_sentence

# Output the representative sentences for each cluster
print(cluster_queries)

{0: 'to be a Borrowing of Revolving Loans or Term Loans ; ( b ) the aggregate amount of such Borrowing ; ( c ) the date of such Borrowing , which shall be a Business Day ; ( d ) whether such Borrowing is to be an ABR Borrowing or a Eurodollar Borrowing ; ( e ) in the case of a Eurodollar Borrowing , the initial Interest Period to be applicable thereto , which shall be a period contemplated by the definition of " Interest Period " ; ( f ) the location and number of the Borrower \'s account to which funds are to be disbursed , which shall comply with the requirements of Section 2.02 ( c ) and ( g ) if and to the extent required under Section 4.01 , that the conditions set forth in clauses ( a ) and ( b ) of Section 4.01 are satisfied as of the date of the notice . If no election as to the Type of Borrowing is specified in any such notice , then the requested Borrowing shall be an ABR Borrowing . If no Interest Period with respect to any requested Eurodollar Revolving Borrowing is specifi

#### Generate queries

In [19]:
import openai

# Set your API key
openai.api_key = "Your OpenAI key"

# Generate queries for each cluster
generated_queries = {}
for cluster_id, representative_sentence in cluster_queries.items():
    prompt = f"""
    I have a dataset of legal sentences labeled as risky. Based on the following examples, generate a query that a lawyer might ask to identify similar risks:
    Query should always in the form of a question. 
    Example:
    Risky Sentence: "If the contract is terminated before completion, the client will incur a 50% penalty."
    Query: "What are the financial risks associated with termination clauses?"

    Now generate a query for this risky sentence:
    "{representative_sentence}"
    """
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}]
    )
    query = response['choices'][0]['message']['content'].strip()
    generated_queries[cluster_id] = query

In [20]:
print(generated_queries)

{0: '"What potential liabilities arise from the structuring of loan agreements under varying types of borrowings?"', 1: '"What potential liabilities arise from retroactive adjustments to interest rates and fees in loan agreements?"', 2: '"What are the risks associated with conditions placed on loan termination and commitment modifications?"', 3: '"What potential liabilities arise from inaccuracies in the calculation of financial ratios in loan agreements?"', 4: 'What are the risks associated with discrepancies between the accounts or records of the Lenders and the Administrative Agent in loan agreements?'}


### Retrieval 

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Function to perform semantic search
def semantic_search(query, embeddings, passages, top_k=5):
    # Generate embedding for the query
    query_embedding = model.encode(query).reshape(1, -1)
    
    # Compute cosine similarities
    similarities = cosine_similarity(query_embedding, embeddings)
    
    # Get top-k results
    top_k_indices = similarities.argsort()[0][-top_k:][::-1]
    results = [(passages[idx], similarities[0][idx]) for idx in top_k_indices]
    return results

In [43]:
embedding_matrix = np.vstack(sample_data["embeddings"])

# Perform retrieval for each query
retrieved_passages_per_query = {}
for query_id, query in generated_queries.items():
    results = semantic_search(query, embedding_matrix, sample_data["cleaned_passage"].tolist(), top_k=5)
    retrieved_passages_per_query[query_id] = results

# Display results
for query_id, results in retrieved_passages_per_query.items():
    print(f"Query {query_id}: {generated_queries[query_id]}")
    for passage, score in results:
        print(f"  - Passage: {passage} | Score: {score}")

Query 0: "What potential liabilities arise from the structuring of loan agreements under varying types of borrowings?"
  - Passage: the amounts of principal and interest payable and paid to such Lender from time to time hereunder . ( c ) The Administrative Agent shall maintain accounts in which it shall record ( i ) the amount of each Loan made hereunder , the Class , Agreed Currency and Type thereof and the Interest Period applicable thereto , ( ii ) the amount of any principal or interest due and payable or to become due and payable from each Borrower to each Lender hereunder and ( iii ) the amount of any sum received by the Administrative Agent hereunder for the account of the Lenders and each Lender 's share thereof . ( d ) The entries made in the accounts maintained pursuant to paragraph ( b ) or ( c ) of this Section shall be prima facie evidence of the existence and amounts of the obligations recorded therein ; 36 provided that the failure of any Lender or the Administrative Age

## Generation

In [44]:
def summarize_results(query, passages):
    # Prepare context for GPT
    context = "\n".join([f"- {p}" for p, _ in passages])
    prompt = f"""
    Query: {query}
    Relevant Passages:
    {context}
    Provide a concise summary of the risks or liabilities identified:
    """
    
    # GPT call
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a legal assistant."},
                  {"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content'].strip()

# Summarize for a specific query
for query_id, results in retrieved_passages_per_query.items():
    summary = summarize_results(generated_queries[query_id], results)
    print(f"Summary for Query {query_id}: {summary}")
    print('\n')


Summary for Query 0: The passage outlines several potential liabilities and risks associated with the structuring of loan agreements under different types of borrowings:

1. **Payment Obligations**: Borrowers have an obligation to repay the principal and interest amounts as recorded by the Administrative Agent. Failure to do so may lead to default.

2. **Account Maintenance Errors**: Errors or failures by lenders or the Administrative Agent in maintaining financial records do not affect the borrower's obligation to repay loans; this could result in disputes over amounts owed.

3. **Prepayment Notices**: Borrowers must follow strict protocols for notifying lenders of prepayments, including specific time frames depending on the type of borrowing. Failure to comply with notice requirements may incur penalties or complicate prepayment processes.

4. **Changes in Law**: Legal changes that render specific loan types (e.g., Eurodollar Loans) unlawful could result in automatic conversions to o